In [ ]:
from http import client
from datetime import date, datetime
import os
import tweepy
import json
import pandas as pd
import time

In [ ]:
CONSUMER_KEY = 'XXXXXXXXXXX'
CONSUMER_SECRET = 'XXXXXXXXXXX'
BEARER_TOKEN = 'XXXXXXXXXXX'

In [ ]:
# Objeto de autenticação
client = tweepy.Client(consumer_key = CONSUMER_KEY, consumer_secret = CONSUMER_SECRET, bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)

In [ ]:
#Funções

def drop_dp(json_file):
    df = pd.read_json(json_file)
    df.drop_duplicates(['texto'], inplace=True)
    with open(json_file, "a+") as output:
        df.to_json(json_file,orient="records")
        
def json_serial(obj):
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))


def filter_query(text, lingua):
    if lingua == 'pt':
        query_set = {'violência', 'leis rígidas', 'leis', 'aborto', 'agressores', 'racismo', 'ladrão', 'bandido', 'feminicídio', 'maioridade penal'}
    elif lingua == 'es':
        query_set = {'racismo', 'gallina amarga', 'tema suarez', 'goleados y golpeados', 'cosas de hombres', 'símios que se hacen llamar hinchas'}
  
    tokens = text.split()
    for count,element in enumerate(tokens):
        if element in query_set:
            return True
        if count == len(tokens) - 1:
            return False

def truncate_utf8_chars(filename, count, ignore_newlines=True):
    """
    Truncates last `count` characters of a text file encoded in UTF-8.
    :param filename: The path to the text file to read
    :param count: Number of UTF-8 characters to remove from the end of the file
    :param ignore_newlines: Set to true, if the newline character at the end of the file should be ignored
    """
    with open(filename, 'rb+') as f:

        size = os.fstat(f.fileno()).st_size

        offset = 1
        chars = 0
        while offset <= size:
            f.seek(-offset, os.SEEK_END)
            b = ord(f.read(1))

            if ignore_newlines:
                if b == 0x0D or b == 0x0A:
                    offset += 1
                    continue

            if b & 0b10000000 == 0 or b & 0b11000000 == 0b11000000:
                # This is the first byte of a UTF8 character
                chars += 1
                if chars == count:
                    # When `count` number of characters have been found, move current position back
                    # with one byte (to include the byte just checked) and truncate the file
                    f.seek(-1, os.SEEK_CUR)
                    f.truncate()
                    return
            offset += 1
            
def pesquisa_tweets(query_str, data_json, i_time, f_time):
    tweets = client.search_all_tweets(query=query_str, max_results = 500, tweet_fields=["created_at","author_id","public_metrics","referenced_tweets"], start_time = i_time, end_time = f_time)
    if (tweets.data != None):
        print(f"Enter\n")
        for tw in tweets.data:
            tweet_autor = tw["author_id"]
            tweet_text = tw["text"]
            tweet_data = tw["created_at"]
            tweet_id = tw["id"]
            tweet_reply_count = tw["public_metrics"]["reply_count"]
            tweet_retweet_count = tw["public_metrics"]["retweet_count"]
            tweet_like_count = tw["public_metrics"]["like_count"]
            with open(data_json, "a+") as output:
                data = {
                "autor":tweet_autor,
                "tw_id":tweet_id,
                "texto":tweet_text,
                "likes":tweet_like_count,
                "replys":tweet_reply_count,
                "retweets": tweet_retweet_count,
                "data":json_serial(tweet_data)}

                output.write(f"{json.dumps(data)},")

def add_pesquisa(query_add, cidade_dict, data_json, i_time, f_time, is_from = True):
    truncate_utf8_chars(data_json, 1)
    with open(data_json, "a+") as output:
        output.write(",")
    
    if is_from == True:
        for i in cidade_dict.values():
            query =  f'(from:{i}) '
            x_query = ''.join((query,query_add))
            pesquisa_tweets(x_query, data_json, i_time, f_time)
    else:
        query = ''
        x_query = ''.join((query,query_add))
        pesquisa_tweets(x_query, data_json, i_time, f_time)

    truncate_utf8_chars(data_json, 1)
    with open(data_json, "a+") as output:
        output.write("]")

#pesquisa de likes por id
def pesquisa_likes(cidade_dict,data_like_json,lingua):
    truncate_utf8_chars(data_like_json, 1)
    with open(data_like_json, "a+") as output:
        output.write(",")

    for j in cidade_dict.values():
        user_like = client.get_liked_tweets(id=f'{j}', max_results = 100, tweet_fields=["created_at","author_id","public_metrics"])
        
        try:
            for like in user_like.data:
                if(filter_query(like['text'],lingua)):
                    tweet_autor_id = like["author_id"]
                    tweet_liked = like["text"]
                    tweet_like_data = like["created_at"]
                    tweet_like_reply_count = like["public_metrics"]["reply_count"]
                    tweet_like_retweet_count = like["public_metrics"]["retweet_count"]
                    tweet_like_likes_count = like["public_metrics"]["like_count"]

                    with open(data_like_json, "a+") as output:
                        data_like = {
                        "autor":tweet_autor_id,
                        "texto":tweet_liked,
                        "likes":tweet_like_likes_count,
                        "retweets":tweet_like_retweet_count,
                        "replys":tweet_like_reply_count,
                        "data":json_serial(tweet_like_data)}
                        output.write(f"{json.dumps(data_like)},\n")
                else:
                    continue
        except:
            continue
    
    truncate_utf8_chars(data_like_json, 1)
    with open(data_like_json, "a+") as output:
        output.write("]")

def pesquisar_mention(metion_json, data_json, s_time, e_time):
    
    truncate_utf8_chars(metion_json, 1)
    with open(metion_json, "a+") as output:
        output.write(",")
    
    with open(data_json) as json_file:
        data_std = json.loads(json_file.read())

        for usr in range(len(data_std)):
            ids_data = data_std[usr]['tw_id']
            mes_antigo = get_month_time(data_std[usr]['data'])
            mes_atual = get_month_time(s_time)
            if (mes_antigo == mes_atual):
                users_mention = client.search_all_tweets(query=f"(is:reply) (conversation_id:{data_std[usr]['tw_id']})",
                                                         start_time = s_time, end_time = e_time,
                                                         max_results = 100, tweet_fields=["author_id","public_metrics"])

                if users_mention.data != None:
                    print("Processando...")
                    for mention in users_mention.data:
                        mention_text = mention["text"]
                        mention_reply_count = mention["public_metrics"]["reply_count"]
                        mention_retweet_count = mention["public_metrics"]["retweet_count"]
                        mention_like_count = mention["public_metrics"]["like_count"]

                        with open(metion_json, "a+") as output_mention:
                            data_mention = {
                            "texto":mention_text,
                            "tw_mencionado":data_std[usr]['texto'],
                            "tw_likes":mention_like_count,
                            "tw_retweet":mention_retweet_count,
                            "tw_replys":mention_reply_count}
                            output_mention.write(f"{json.dumps(data_mention)},\n")
                else:
                    print("Vazio")
            else:
                pass
    
    truncate_utf8_chars(metion_json, 1)
    with open(metion_json, "a+") as output:
        output.write("]")